classifiers: nn.crossentropyloss = -log-likelihood --- use for logp(y) and -logq(y/x) for support <br>
kl-div: <br>
reconstr-loss: set reduction to none and then take mean of losses per image in the total batch. This gives reconstr-loss per image for further computation<br> 

In [1]:
import sys
import random
import numpy as np
import torch
import learn2learn as l2l
import pandas as pd

from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from PIL.Image import LANCZOS
from config import *

import json
import copy
import math

from torch.distributions.multivariate_normal import MultivariateNormal


In [2]:
torch.cuda.is_available()


True

In [3]:
sys.path.append('/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn')
sys.path

['/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn/src',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles',
 '/home/anuj/.vscode/extensions/ms-toolsai.jupyter-2021.8.1236758218/pythonFiles/lib/python',
 '/home/anuj/anaconda3/envs/torch/lib/python38.zip',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/lib-dynload',
 '',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/datasets-1.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/IPython/extensions',
 '/home/anuj/.ipython',
 '/home/anuj/Desktop/Work/TU_Delft/research/implement/learning_to_meta-learn']

In [4]:
#from data.loaders import Omniglotmix, MiniImageNet
from data.taskers import gen_tasks
from src.zoo.archs import CVAE, LVAE
from src.zoo.lpo_utils import proto_distr, classify, set_sets, accuracy, inner_adapt_lpo, setup, kl_div

#from src.utils import Profiler

In [5]:
train_tasks = gen_tasks(dataname='miniimagenet', root='../../mini_imagenet', mode='train', n_ways=5, k_shots=1, q_shots=1, image_transforms=None)

In [5]:
classes = list(range(1623))
random.shuffle(classes)
image_transforms = transforms.Compose([transforms.Resize(28, interpolation=LANCZOS),
                                                    transforms.ToTensor(),
                                                    lambda x: 1.0 - x,
                                                ])
train_tasks = gen_tasks('omniglot', '/home/anuj/Desktop/Work/TU_Delft/research/implement/omniglot', image_transforms=image_transforms, n_ways=5, k_shots=5, q_shots=5, classes=classes[:1100])

/home/anuj/anaconda3/envs/torch/lib/python3.8/site-packages/torchvision/transforms/transforms.py:257: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [5]:
train_tasks, valid_tasks, test_tasks, learner, learner_temp, embedder = setup('miniimagenet', '../../mini_imagenet', 5, 3, 7, 5, 3, 7, 'cpu')

In [6]:
task = train_tasks.sample()
device = 'cpu'
n_ways = 5
k_shots = 3
q_shots = 7

In [7]:
data, labels = task
data, labels = data.to(device), labels.to(device)
#data, labels = data.squeeze(0), labels.squeeze(0)
sort = torch.sort(labels)
data = data.squeeze(0)[sort.indices].squeeze(0)
labels = labels.squeeze(0)[sort.indices].squeeze(0)
total = n_ways * (k_shots + q_shots)
queries_index = np.zeros(total)

# Extracting the query datums from the entire task set
for offset in range(n_ways):
    queries_index[np.random.choice(
        k_shots+q_shots, q_shots, replace=False) + ((k_shots + q_shots)*offset)] = True
support = data[np.where(queries_index == 0)]
support_labels = labels[np.where(queries_index == 0)]
queries = data[np.where(queries_index == 1)]
queries_labels = labels[np.where(queries_index == 1)]


In [8]:
queries_labels

tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
        3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4])

In [9]:

y_support = F.one_hot(support_labels, num_classes=n_ways)
y_queries = torch.tensor(range(n_ways))
y_queries = y_queries.repeat(n_ways*q_shots)
y_queries = F.one_hot(y_queries, num_classes=n_ways)
qs = queries.repeat_interleave(n_ways, dim=0)

support, qs = embedder(support), embedder(qs)


In [10]:
y_support.shape

torch.Size([15, 5])

In [11]:
support_cap, support_mu, support_log_var = learner(support, y_support)
support_mu

tensor([[ 1.3663e-01,  2.4987e-02,  5.5654e-03, -9.8920e-03, -7.6280e-02,
         -7.1484e-02, -1.0899e-01,  1.5873e-01, -1.1427e-01, -6.6355e-02,
         -1.4163e-01,  1.7571e-02,  6.9741e-02,  1.1391e-01,  9.5002e-02,
          2.6588e-02, -7.3195e-02,  2.3054e-01, -2.5127e-02, -3.4086e-02,
         -4.2776e-02, -9.1617e-02,  7.0079e-03,  4.8824e-02, -9.7244e-02,
          6.5436e-02, -4.2479e-02,  1.9061e-01,  7.1781e-02,  2.2788e-01,
          7.2408e-02,  2.1769e-02, -2.3030e-02,  2.0740e-01,  1.3754e-01,
          8.9016e-02, -2.4195e-01, -2.6483e-02, -2.6474e-01,  1.8368e-02,
         -6.4890e-02, -5.7888e-02,  9.3043e-02,  8.5093e-02,  4.8365e-02,
         -5.6663e-02,  4.0189e-02, -2.8450e-02, -1.3092e-01,  1.8675e-01,
         -1.3265e-01, -7.3903e-03, -1.6466e-01,  3.8370e-02, -2.8796e-01,
         -5.6105e-02, -4.8747e-02,  1.3053e-01, -3.3306e-02, -5.2846e-02,
         -2.6578e-02,  6.0663e-02, -4.4995e-02,  5.4612e-02],
        [ 1.6521e-01,  2.6701e-02, -1.0535e-02, -3

In [12]:
# Building Prototypical distributions
proto_mu, proto_var = proto_distr(
    support_mu, support_log_var, n_ways, k_shots, 'precision_weighted')


In [13]:
proto_var.shape

torch.Size([5, 64])

In [14]:
proto_mu.shape

torch.Size([5, 64])

In [15]:

# Forward pass on the Query datums
queries_cap, queries_mu, queries_log_var = learner(qs, y_queries)
queries_cap

tensor([[-0.0181,  0.1320,  0.1416,  ...,  0.1404,  0.1503, -0.0212],
        [ 0.0554,  0.1279,  0.0390,  ...,  0.0312,  0.1333,  0.0928],
        [ 0.1802,  0.1755,  0.0260,  ...,  0.0782,  0.0141, -0.0260],
        ...,
        [ 0.1379,  0.2035,  0.0282,  ...,  0.1016,  0.0877,  0.0401],
        [-0.0352,  0.1227,  0.0220,  ...,  0.0677,  0.1188, -0.0588],
        [ 0.1649,  0.1123,  0.1100,  ...,  0.0597,  0.1080, -0.0852]],
       grad_fn=<AddmmBackward>)

In [21]:
support_mu.unsqueeze(1).expand(a,b,-1)

tensor([[[ 0.1366,  0.0250,  0.0056,  ...,  0.0607, -0.0450,  0.0546],
         [ 0.1366,  0.0250,  0.0056,  ...,  0.0607, -0.0450,  0.0546],
         [ 0.1366,  0.0250,  0.0056,  ...,  0.0607, -0.0450,  0.0546],
         [ 0.1366,  0.0250,  0.0056,  ...,  0.0607, -0.0450,  0.0546],
         [ 0.1366,  0.0250,  0.0056,  ...,  0.0607, -0.0450,  0.0546]],

        [[ 0.1652,  0.0267, -0.0105,  ...,  0.0742, -0.0504,  0.0548],
         [ 0.1652,  0.0267, -0.0105,  ...,  0.0742, -0.0504,  0.0548],
         [ 0.1652,  0.0267, -0.0105,  ...,  0.0742, -0.0504,  0.0548],
         [ 0.1652,  0.0267, -0.0105,  ...,  0.0742, -0.0504,  0.0548],
         [ 0.1652,  0.0267, -0.0105,  ...,  0.0742, -0.0504,  0.0548]],

        [[ 0.1410, -0.0434, -0.0258,  ...,  0.0309, -0.0440,  0.0661],
         [ 0.1410, -0.0434, -0.0258,  ...,  0.0309, -0.0440,  0.0661],
         [ 0.1410, -0.0434, -0.0258,  ...,  0.0309, -0.0440,  0.0661],
         [ 0.1410, -0.0434, -0.0258,  ...,  0.0309, -0.0440,  0.0661],
  

In [23]:
a = queries_mu.shape[0]
b = proto_mu.shape[0]

# delta.t * Q**-1 * delta

delta = queries_mu.unsqueeze(1).expand(a,b,-1) - proto_mu.unsqueeze(0).expand(a,b,-1)
logits = -0.5 * torch.mul(torch.mul(delta, proto_var.unsqueeze(0).expand(a,b,-1)**(-1)), delta).sum(dim=-1)
logits.shape

torch.Size([175, 5])

In [40]:

support_logits = classify(
    mu_p=proto_mu, var_p=proto_var, mu_datums=support_mu)
queries_logits = classify(
    mu_p=proto_mu, var_p=proto_var, mu_datums=queries_mu)
support_logits

tensor([[-23.6021, -23.6018, -23.6142, -23.5726, -23.5880],
        [-23.5873, -23.5628, -23.4816, -23.4948, -23.4916],
        [-23.5813, -23.5522, -23.5197, -23.5076, -23.5279],
        [-23.5949, -23.5359, -23.5181, -23.5067, -23.5110],
        [-23.5810, -23.5382, -23.5119, -23.5014, -23.5085],
        [-23.6005, -23.5409, -23.5303, -23.5203, -23.5227],
        [-23.6427, -23.6106, -23.4794, -23.5159, -23.4984],
        [-23.6293, -23.5917, -23.4693, -23.5036, -23.4954],
        [-23.6215, -23.5772, -23.4727, -23.4979, -23.4884],
        [-23.6039, -23.5669, -23.5206, -23.5002, -23.5154],
        [-23.5838, -23.5446, -23.4925, -23.4868, -23.4953],
        [-23.6467, -23.6168, -23.4848, -23.5051, -23.4993],
        [-23.6246, -23.5816, -23.5140, -23.5177, -23.4837],
        [-23.6001, -23.5622, -23.4982, -23.4963, -23.4792],
        [-23.6877, -23.6445, -23.4968, -23.5423, -23.5072]],
       grad_fn=<SumBackward1>)

In [43]:

# adding up the losses
ce_loss = torch.nn.CrossEntropyLoss(reduction='none')
reconstruction_loss = nn.MSELoss(reduction='none')

In [49]:

L_support = -reconstruction_loss(support_cap, support).view(support.shape[0], -1).mean(dim=1) - ce_loss(
    F.softmax(torch.ones_like(y_support).float(), dim=1), torch.argmax(y_support, dim=1)) - kl_div(support_mu, support_log_var)  # = -L(x_s, y_s)
L_support

tensor([-2.5833, -3.0038, -2.8290, -2.5975, -2.6868, -2.6668, -3.2976, -3.0719,
        -2.9612, -2.6229, -2.8146, -3.3088, -2.8344, -2.8077, -3.4454],
       grad_fn=<SubBackward0>)

In [65]:

L_queries = -reconstruction_loss(queries_cap, qs).view(qs.shape[0], -1).mean(dim=1) - ce_loss(
    F.softmax(torch.ones_like(y_queries).float(), dim=1), torch.argmax(y_queries, dim=1)) - kl_div(queries_mu, queries_log_var)  # = -L(x_q, y_q)
L_queries

tensor([-2.7519, -2.7516, -2.7609, -2.7493, -2.7607, -2.6219, -2.6133, -2.6251,
        -2.6283, -2.6194, -2.7640, -2.7708, -2.7782, -2.7780, -2.7817, -2.7049,
        -2.6945, -2.7047, -2.7047, -2.7028, -2.6130, -2.6090, -2.6101, -2.6148,
        -2.6060, -2.7029, -2.7020, -2.7052, -2.7152, -2.7026, -2.6540, -2.6356,
        -2.6403, -2.6457, -2.6380, -2.5660, -2.5774, -2.5702, -2.5704, -2.5642,
        -2.8104, -2.8126, -2.8134, -2.8128, -2.8140, -2.8794, -2.8642, -2.8698,
        -2.8728, -2.8693, -2.9672, -2.9540, -2.9480, -2.9431, -2.9446, -2.9832,
        -2.9816, -2.9821, -2.9812, -2.9750, -2.5828, -2.5861, -2.5875, -2.5781,
        -2.5856, -2.8765, -2.8712, -2.8730, -2.8725, -2.8709, -2.7327, -2.7147,
        -2.7157, -2.7129, -2.7193, -3.5113, -3.4952, -3.4876, -3.4819, -3.4779,
        -2.5509, -2.5536, -2.5512, -2.5452, -2.5477, -2.7118, -2.7136, -2.7242,
        -2.7072, -2.7143, -3.3958, -3.3839, -3.3937, -3.3829, -3.3937, -2.7552,
        -2.7553, -2.7500, -2.7622, -2.76

In [63]:
yq = torch.argmax(y_queries, dim=1)
yq.view(n_ways*q_shots, n_ways).t()

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
         3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
         4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]])

In [71]:
L_queries.view(n_ways*q_shots, n_ways).shape

torch.Size([35, 5])

In [76]:
U_queries = torch.mul(F.softmax(queries_logits, dim=1)[
                        ::5, ], L_queries.view(n_ways*q_shots, n_ways)).sum(dim=1) - 0.1*torch.sum(torch.mul(F.softmax(queries_logits, dim=1)[
                            ::5, ], torch.log(F.softmax(queries_logits, dim=1)[
                                ::5, ])), dim=1)
U_queries

tensor([-2.5940, -2.4607, -2.6137, -2.5415, -2.4497, -2.5448, -2.4817, -2.4088,
        -2.6518, -2.7102, -2.7902, -2.8197, -2.4231, -2.7119, -2.5580, -3.3294,
        -2.3888, -2.5534, -3.2292, -2.5962, -3.4405, -2.7984, -3.1919, -2.4627,
        -3.1805, -2.8073, -2.7704, -2.5756, -2.5687, -2.7281, -2.4993, -3.1080,
        -3.1591, -2.5205, -3.0378], grad_fn=<SubBackward0>)

In [79]:
alpha = 0.1*(q_shots/k_shots)
J_alpha = -L_support.mean() - U_queries.mean() + alpha * \
    ce_loss(support_logits, torch.argmax(y_support, dim=1)).mean()
J_alpha

tensor(6.0066, grad_fn=<AddBackward0>)

In [5]:
train_tasks, valid_tasks, test_tasks, learner, learner_temp, embedder = setup('miniimagenet', '../../mini_imagenet', 5, 3, 7, 5, 3, 7, 'cpu')

device = 'cpu'
n_ways = 5
k_shots = 3
q_shots = 7

In [6]:
loss = nn.MSELoss(reduction='none')
## Training ##
meta_train_loss = []
meta_valid_loss = []
meta_train_acc = []
meta_valid_acc = []
learner.train()


LVAE(
  (encoder): LEncoder(
    (net): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=128, bias=True)
      (3): ReLU()
    )
    (h1): Linear(in_features=128, out_features=64, bias=True)
    (h2): Linear(in_features=128, out_features=64, bias=True)
  )
  (decoder): LDecoder(
    (net): Sequential(
      (0): Linear(in_features=69, out_features=128, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
      (4): Linear(in_features=256, out_features=512, bias=True)
    )
  )
)

In [7]:
learner_temp_state = copy.deepcopy(learner.state_dict())
learner_temp.load_state_dict(learner_temp_state)
opt = optim.Adam(learner_temp.parameters(), 0.001)

In [8]:
[p for p in learner_temp.parameters()]

[Parameter containing:
 tensor([[ 0.0108,  0.0243,  0.0185,  ..., -0.0164,  0.0236,  0.0374],
         [ 0.0068,  0.0064, -0.0116,  ...,  0.0058,  0.0020,  0.0191],
         [-0.0327,  0.0254, -0.0249,  ..., -0.0022,  0.0181,  0.0160],
         ...,
         [ 0.0349, -0.0221,  0.0433,  ...,  0.0178,  0.0242,  0.0199],
         [ 0.0255,  0.0358, -0.0036,  ..., -0.0374,  0.0353, -0.0067],
         [-0.0255,  0.0315,  0.0118,  ..., -0.0030,  0.0020,  0.0005]],
        requires_grad=True),
 Parameter containing:
 tensor([ 1.3028e-02, -3.7853e-02, -4.3363e-02,  2.5034e-02,  2.1001e-02,
          3.0429e-02, -1.9690e-02, -4.4064e-02,  7.8055e-03,  1.2786e-02,
         -4.1244e-02, -4.0994e-02,  1.8539e-02, -3.3859e-03, -5.5976e-03,
         -2.7980e-02, -2.4508e-02, -1.3778e-02, -3.1488e-02,  2.7404e-02,
          4.2747e-02, -1.2746e-02,  2.0945e-02,  1.1789e-02,  1.0178e-02,
          5.0657e-03, -3.2606e-02,  7.6419e-03,  1.0938e-02,  4.3282e-02,
         -4.0774e-03, -3.9221e-02,  2.79

In [9]:
[p for p in learner.parameters()]

[Parameter containing:
 tensor([[ 0.0108,  0.0243,  0.0185,  ..., -0.0164,  0.0236,  0.0374],
         [ 0.0068,  0.0064, -0.0116,  ...,  0.0058,  0.0020,  0.0191],
         [-0.0327,  0.0254, -0.0249,  ..., -0.0022,  0.0181,  0.0160],
         ...,
         [ 0.0349, -0.0221,  0.0433,  ...,  0.0178,  0.0242,  0.0199],
         [ 0.0255,  0.0358, -0.0036,  ..., -0.0374,  0.0353, -0.0067],
         [-0.0255,  0.0315,  0.0118,  ..., -0.0030,  0.0020,  0.0005]],
        requires_grad=True),
 Parameter containing:
 tensor([ 1.3028e-02, -3.7853e-02, -4.3363e-02,  2.5034e-02,  2.1001e-02,
          3.0429e-02, -1.9690e-02, -4.4064e-02,  7.8055e-03,  1.2786e-02,
         -4.1244e-02, -4.0994e-02,  1.8539e-02, -3.3859e-03, -5.5976e-03,
         -2.7980e-02, -2.4508e-02, -1.3778e-02, -3.1488e-02,  2.7404e-02,
          4.2747e-02, -1.2746e-02,  2.0945e-02,  1.1789e-02,  1.0178e-02,
          5.0657e-03, -3.2606e-02,  7.6419e-03,  1.0938e-02,  4.3282e-02,
         -4.0774e-03, -3.9221e-02,  2.79

In [24]:
ttask = train_tasks.sample()
support, y_support, queries, qs, y_queries, queries_labels = set_sets(
    ttask, n_ways, k_shots, q_shots, embedder, device)

In [27]:
# Running inner adaptation loop
for i in range(50):
    opt.zero_grad()
    evaluation_loss, query_preds = inner_adapt_lpo(
        support, y_support, qs, y_queries, learner_temp, loss, n_ways, k_shots, q_shots, 3/7, 1)
    evaluation_loss.backward()
    opt.step()

meta_train_loss.append(evaluation_loss.item())
evaluation_accuracy = accuracy(query_preds[::n_ways,], queries_labels)
meta_train_acc.append(evaluation_accuracy.item())


In [28]:
meta_train_acc

[0.4285714328289032,
 0.4285714328289032,
 0.4000000059604645,
 0.2571428716182709,
 0.2857142984867096,
 0.22857142984867096,
 0.11428571492433548,
 0.20000000298023224]

In [33]:
for p in learner_temp.parameters():
    print(p.grad.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.0018, -0.0018, -0.0015,  ..., -0.0019, -0.0015, -0.0013],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])
tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0034,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000, -0.0065,  0.0000,  0.0000,  0.0000,
        -0.0007,  0.0000,  0.0000,  0.0000, -0.0061,  0.0000,  0.0020, -0.0164,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
      

In [69]:
- 0.5 * np.log(2 * np.pi)

-0.9189385332046727